# Multi Class Drone State Classification Model_v0.1

This model is designed to classify the state of a drone into multiple categories based on its sensor data using Binary Relevance with Logistic Regression as the base classifier.


### Dataset Loading and Exploration

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
data = pd.read_csv('C:\Meet\Projects\Project_6_Icarus_Autonomous Drone Navigation Research\Project-Icarus\Drone Trajectory Classification Model\Dataset\Drone onboard multi-modal sensor dataset for complex outdoor scenarios.csv')

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\meetb\AppData\Local\Temp\ipykernel_23860\3566205407.py:1: SyntaxWarning: invalid escape sequence '\M'
  data = pd.read_csv('C:\Meet\Projects\Project_6_Icarus_Autonomous Drone Navigation Research\Project-Icarus\Drone Trajectory Classification Model\Dataset\Drone onboard multi-modal sensor dataset for complex outdoor scenarios.csv')


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276057 entries, 0 to 276056
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   flight                 276057 non-null  int64  
 1   seq                    276057 non-null  int64  
 2   uid                    276057 non-null  object 
 3   timestamp              276057 non-null  int64  
 4   flight_logic_state     276057 non-null  object 
 5   wind_speed             276057 non-null  float64
 6   wind_angle             276057 non-null  int64  
 7   battery_voltage        276057 non-null  int64  
 8   battery_current        276057 non-null  int64  
 9   position_x             276057 non-null  float64
 10  position_y             276057 non-null  float64
 11  position_z             276057 non-null  float64
 12  altitude               276057 non-null  float64
 13  orientation_x          276057 non-null  float64
 14  orientation_y          276057 non-nu

In [14]:
data.describe()

,flight,seq,timestamp,wind_speed,wind_angle,battery_voltage,battery_current,position_x,position_y,position_z,...,linear_acceleration_y,linear_acceleration_z,payload,yaw,flight_1,IDLE_HOVER,ASCEND,TURN,HMSL,DESCEND
count,276057.000000,276057.000000,2.760570e+05,276057.000000,276057.000000,276057.000000,276057.000000,276057.000000,276057.000000,276057.000000,...,276057.000000,276057.000000,2.760570e+05,276057.000000,276057.0,276057.000000,276057.000000,276057.000000,276057.000000,276057.000000
mean,9.780795,138029.000000,1.721888e+09,15.240006,208.525609,45786.548488,16564.132846,33.414741,35.145350,59.168713,...,-0.001156,-0.000886,4.710000e-01,-0.344682,1.0,0.333025,0.018706,0.185125,0.546862,0.044900
std,5.753696,79690.935968,4.207535e+05,39.005961,111.662821,1955.305450,5138.903166,0.000662,0.000503,25.289246,...,0.777099,0.320406,2.775563e-16,2.012692,0.0,0.471296,0.135486,0.388399,0.497800,0.207085
min,1.000000,1.000000,1.720598e+09,-99.699997,0.000000,42902.000000,-44.000000,33.411964,35.143097,-2.439651,...,-4.998427,-4.656783,4.710000e-01,-3.141558,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,69015.000000,1.721723e+09,3.910000,99.000000,44229.000000,17197.000000,33.414753,35.145252,69.848740,...,-0.215903,-0.075119,4.710000e-01,-2.410089,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,138029.000000,1.721808e+09,10.720000,266.000000,45418.000000,18040.000000,33.414925,35.145538,69.985550,...,-0.002208,-0.000006,4.710000e-01,-0.677118,1.0,0.000000,0.000000,0.000000,1.000000,0.000000
75%,15.000000,207043.000000,1.722240e+09,34.849998,296.000000,46861.000000,18766.000000,33.415112,35.145649,70.028336,...,0.217362,0.076748,4.710000e-01,1.595755,1.0,1.000000,0.000000,0.000000,1.000000,0.000000
max,20.000000,276057.000000,1.722505e+09,99.989998,360.000000,52210.000000,25972.000000,33.415363,35.145779,101.204239,...,5.311280,7.422338,4.710000e-01,3.141569,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


## Dataset Preprocessing

In [15]:
# Check for class imbalance and multi-label instances
data[['IDLE_HOVER','ASCEND','TURN','HMSL','DESCEND']].sum(axis=1).value_counts()

1    243191
2     30226
3      2640
Name: count, dtype: int64

### Seperating Features and Labels

In [16]:
label_cols = ['IDLE_HOVER','ASCEND','TURN','HMSL','DESCEND']
y = data[label_cols]

### Dropping Identifier and Metadata Columns

In [17]:
drop_cols = ['flight', 'seq', 'uid', 'timestamp'] + label_cols
X = data.drop(columns=drop_cols)

### Checking for columns with constant values

In [18]:
X.describe().loc['std'].sort_values()

flight_1                 0.000000e+00
payload                  2.775563e-16
position_y               5.029859e-04
position_x               6.622490e-04
orientation_y            4.211050e-02
orientation_x            4.533615e-02
angular_x                9.356194e-02
angular_y                2.152616e-01
linear_acceleration_z    3.204061e-01
angular_z                3.896060e-01
orientation_w            5.183924e-01
orientation_z            5.810358e-01
velocity_z               6.593509e-01
linear_acceleration_y    7.770990e-01
linear_acceleration_x    8.473555e-01
yaw                      2.012692e+00
velocity_y               2.321745e+00
velocity_x               2.528063e+00
altitude                 2.528925e+01
position_z               2.528925e+01
wind_speed               3.900596e+01
wind_angle               1.116628e+02
battery_voltage          1.955305e+03
battery_current          5.138903e+03
Name: std, dtype: float64

In [19]:
# Checking for columns with constant values
constant_cols = X.columns[X.nunique() <= 1]
print("Columns with constant values:", constant_cols.tolist())
X = X.drop(columns=constant_cols)

Columns with constant values: ['flight_logic_state', 'payload', 'flight_1']


### Scaling Numeric Features

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Inspect dtypes to confirm non-numeric columns
print("Non-numeric columns:", X.select_dtypes(exclude=['number']).columns.tolist())

# Keep numeric features and clean them
X_num = X.select_dtypes(include=['number']).copy()
X_num = X_num.replace([np.inf, -np.inf], np.nan)

imputer = SimpleImputer(strategy='median')
X_num_imputed = imputer.fit_transform(X_num)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num_imputed)

Non-numeric columns: []


### Train Test Split

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert labels to numpy
y_array = y.values.astype(int)

# Train-test split (stratifying by number of active labels)
label_density = y_array.sum(axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_array, test_size=0.2, random_state=42, stratify=label_density
)

### Logistic Regression Model

In [22]:
def sigmoid(z):
    z = np.clip(z, -50, 50)
    return 1 / (1 + np.exp(-z))

class LogisticRegressionScratch:
    def __init__(self, n_features, lr=0.01, epochs=3):
        self.w = np.zeros(n_features)
        self.b = 0.0
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        for epoch in range(self.epochs):
            total_loss = 0
            for i in range(X.shape[0]):  # 🟡 batch (per-sample update)
                xi, yi = X[i], y[i]
                z = np.dot(self.w, xi) + self.b
                pred = sigmoid(z)
                error = pred - yi
                self.w -= self.lr * (error * xi)
                self.b -= self.lr * error
                total_loss += -(yi * np.log(pred+1e-9) + (1-yi) * np.log(1-pred+1e-9))
            print(f"Epoch {epoch+1}/{self.epochs} Loss = {total_loss / X.shape[0]}")

    def predict(self, X, threshold=0.5):
        return (sigmoid(np.dot(X, self.w) + self.b) >= threshold).astype(int)


### Training One-vs-Rest Logistic Regression Models

In [23]:
label_names = ['IDLE_HOVER','ASCEND','TURN','HMSL','DESCEND']
models = []

for i, label in enumerate(label_names):
    print(f"\nTraining for label: {label}")
    model = LogisticRegressionScratch(n_features=X_train.shape[1], lr=0.001, epochs=3)
    model.fit(X_train, y_train[:, i])
    models.append(model)



Training for label: IDLE_HOVER
Epoch 1/3 Loss = 0.45234865881558256
Epoch 2/3 Loss = 0.44785998365183205
Epoch 3/3 Loss = 0.44778764085279954

Training for label: ASCEND
Epoch 1/3 Loss = 0.035269956931548184
Epoch 2/3 Loss = 0.010235991301477036
Epoch 3/3 Loss = 0.008748073947774373

Training for label: TURN
Epoch 1/3 Loss = 0.40342167528166595
Epoch 2/3 Loss = 0.39770585177652584
Epoch 3/3 Loss = 0.39762886166319106

Training for label: HMSL
Epoch 1/3 Loss = 0.47778070104913267
Epoch 2/3 Loss = 0.4742313609438164
Epoch 3/3 Loss = 0.4742164461550996

Training for label: DESCEND
Epoch 1/3 Loss = 0.05810740690325681
Epoch 2/3 Loss = 0.02663472229676652
Epoch 3/3 Loss = 0.024070129767652795


### Prediction and Evaluation

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_pred = np.zeros_like(y_test)

for i, model in enumerate(models):
    preds = model.predict(X_test)
    y_pred[:, i] = preds
    acc = np.mean(preds == y_test[:, i])
    prec = precision_score(y_test[:, i], preds, zero_division=0)
    rec = recall_score(y_test[:, i], preds, zero_division=0)
    f1 = f1_score(y_test[:, i], preds, zero_division=0)
    print(f"{label_names[i]} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

# Multi-label metrics
hamming = np.mean(y_pred != y_test)
subset_acc = accuracy_score(y_test, y_pred)
micro_f1 = f1_score(y_test, y_pred, average='micro', zero_division=0)
macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
weighted_f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"\nOverall Hamming Loss: {hamming:.4f}")
print(f"Subset Accuracy: {subset_acc:.4f}")
print(f"Micro F1-score: {micro_f1:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")
print(f"Weighted F1-score: {weighted_f1:.4f}")

IDLE_HOVER - Accuracy: 0.8042, Precision: 0.7726, Recall: 0.5834, F1: 0.6648
ASCEND - Accuracy: 0.9983, Precision: 0.9639, Recall: 0.9454, F1: 0.9545
TURN - Accuracy: 0.8442, Precision: 0.7782, Recall: 0.2225, F1: 0.3461
HMSL - Accuracy: 0.8131, Precision: 0.8007, Recall: 0.8754, F1: 0.8364
DESCEND - Accuracy: 0.9944, Precision: 0.9626, Recall: 0.9129, F1: 0.9371

Overall Hamming Loss: 0.1092
Subset Accuracy: 0.6209
Micro F1-score: 0.7390
Macro F1-score: 0.7478
Weighted F1-score: 0.7114


### Testing output

In [26]:
# Predict all labels for a given sample
sample = X_test[1].reshape(1, -1)
pred_vector = [model.predict(sample)[0] for model in models]
print("Actual labels:", y_test[1])
print("Multi-label prediction:", pred_vector)

Actual labels: [0 0 0 1 0]
Multi-label prediction: [0, 0, 0, 1, 0]
